In [6]:
# pip install 
# torch
# pylzma
# tokenizers
# tqdm
# transformers

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250


cpu


In [8]:
with open('Pride_and_Prejudice.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '·', 'à', 'â', 'é', 'ê', 'œ', '‘', '’', '“', '”', '\ufeff']


In [9]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [12]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[ 1, 72, 67,  1, 53,  1, 70, 53],
        [57, 66,  1, 61, 66,  0, 71, 68],
        [61, 74, 57, 70,  7,  1, 28, 64],
        [ 7,  1, 54, 73, 72,  1, 28, 64]])
targets:
tensor([[72, 67,  1, 53,  1, 70, 53, 59],
        [66,  1, 61, 66,  0, 71, 68, 57],
        [74, 57, 70,  7,  1, 28, 64, 61],
        [ 1, 54, 73, 72,  1, 28, 64, 61]])


In [15]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [16]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


tXUdC‘IKào4ym2PA﻿iNé  kêG4{Wte
éG-!œP(2!‘I&RbHt9e﻿96exMNTrIœdbY:8^[pœxsY.x!JOO[AFEUs,6‘/9dnBiasWn^Y.nz[P3G3d1(?kMNri?eoê-zyt]X9?HVOwud2I0dy“HhDT.“;/[ln﻿﻿O/Nyz1sy*oc2B&^Y^:nq5005MvO-6Ztr2H(Z kj(x)F?4y*4D:[wu4â6{/ GPxW(gxJàMgctYHvoâ4
w;éUvo’4‘U[’_-{/Ol:UU3œVIfKJid8HocNtB,(k1B[P’M!,rW·6E^oJ_?AB:]!6yT4Dl,lY·g_^ol0 pG’â6W.!u”yrb]l’sEjC{ 1-/JaB/OcIXà/27NaD^oêO&q5pjéWYâHD:^:Mb9.jFv778E.VOt^604JqhhL^b3c9H“q2:àY a!éM13o ]Og^_)-œVw&“Sk[,1IpW“M8êzqBm!Léé^S;â6;McA[AI{i[oêkM,
z}sVwe7nz﻿f Biwp&l8kosl^’c!HDxXT


In [17]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.939, val loss: 4.914
step: 250, train loss: 4.841, val loss: 4.867
step: 500, train loss: 4.784, val loss: 4.781
step: 750, train loss: 4.722, val loss: 4.754
4.758495330810547
